In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,  GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np

import cv2
import os
import glob


In [ ]:
path_happy='Data/happy'
path_neut='Data/Neutral'
# path_unkno='image/unknown'

la=len(os.listdir(path_happy))
lb=len(os.listdir(path_neut))     
# lc=len(os.listdir(path_unkno)) 

print('Happy 경로에 저장된 파일의 개수:'+str(la))
print('Neutral 경로에 저장된 파일의 개수:'+str(lb))
# print('Chris 경로에 저장된 파일의 개수:'+str(lc))

In [ ]:
xsize=256
ysize=256

In [ ]:
x=[]
y=[]

from keras.preprocessing import image
import numpy as np

def read_dir(path, label):
    
    files = glob.glob(path + "\\/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "\\/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            
            y.append(label)
            x.append(img_tr)
        except:
            pass

In [ ]:
read_dir(path_happy, 0) #'DanielCraig')
read_dir(path_neut, 1) #'ChrisEvans')

In [ ]:
x=np.array(x)
y=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

input_shape = (xsize,ysize, 3)

batch_size = 32
num_classes = 2
epochs = 15

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# model= Sequential()
# model.add(Conv2D(32, kernel_size=(6, 6), activation='relu',input_shape=(256, 256,3)))
# model.add(MaxPooling2D(pool_size=(4, 4)))

# model.add(Conv2D(32, kernel_size=(6, 6), activation='relu',input_shape=(256, 256,3)))
# model.add(MaxPooling2D(pool_size=(4, 4)))

# model.add(Conv2D(32, kernel_size=(4, 4), activation='relu', input_shape=(256, 256,3))) # 32
# model.add(MaxPooling2D(pool_size=(4, 4)))

# model.add(Flatten())
# model.add(Dense(8, activation='relu')) #layers.Dense() : densely-connected NN layer
# model.add(Dense(num_classes, activation='softmax')) 

# model.summary()

In [ ]:
from keras.applications import ResNet50
from keras.models import Model

img_width =256
img_height = 256
img_channel = 3

model = Sequential()

from keras.models import Input

input = Input(shape=(256, 256, 3))
model = ResNet50(input_tensor=input, include_top=False, weights=None, pooling='max')
xa = model.output
xb = Dense(2, activation='softmax', name='soft')(xa)
model = Model(model.input, xb)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1, validation_split=0.1)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])